# Stream to Features
  --------------------------------------------------------------------

Receive a stream of events from `incoming-events-stream`, enrich specific events and update a set of aggregations on the data. The output data is stored to an aggregation table called `feature-table` and a new event that includes the calculated features is written to `serving-stream`


Enrich the relevant events with socioeconomic data by looking up the enrichment table.<br>
During the feature calculation, we calculate sum, mean, count and variance for the 3 amount fields (`amount`, `bet_amount` and `win_amount` for `new_purchases`, `new_bet` and `new_win` respectively). This results with the following list of fields:

- purchase_sum
- purchase_mean
- purchase_count
- purchase_var
- bet_sum
- bet_mean
- bet_count
- bet_var
- win_sum
- win_mean
- win_count
- win_var

You can change the incoming events and the generated features by customizing the methods below.

## Create and Test a Local Function 

[Nuclio](https://nuclio.io/) is a high-performance open-source and managed serverless framework, which is available as a predefined tenant-wide platform service (`nuclio`).
The demo uses Nuclio to create and deploy serverless functions.
Therefore, you need to import the Nuclio package and configure Nuclio for your project.

The platform's Jupyter Notebook service preinstalls the [nuclio-jupyter SDK](https://github.com/nuclio/nuclio-jupyter/blob/master/README.md) for creating and deploying Nuclio functions with Python and Jupyter Notebook.
The tutorial uses the Nuclio magic commands and annotation comments of this SDK to automate function code generation.
The magic commands are initialized when you import the `nuclio` package.<br>
The `%nuclio` magic commands are used to run Nuclio commands from Jupyter notebooks (`%nuclio <Nuclio command>`).
You can also use `%%nuclio` at the start of a cell to identify the entire cell as containing Nuclio code.
The magic commands are initialized when you import the `nuclio` package.<br>
The `# nuclio: start-code`, `# nuclio: end-code`, and `# nuclio: ignore` section-marker annotations notify Nuclio of the beginning or end of code sections.
Nuclio ignores all notebook code before a `# nuclio: start-code` marker or after an `# nuclio: end-code` marker.
Nuclio translates all other notebook code sections into function code, except for sections that are marked with the `# nuclio: ignore` marker.

### Import Nuclio

The following code imports the `nuclio` Python package.

In [ ]:
import nuclio

#### Configure Nuclio

The following code uses the `# nuclio: start-code` marker to instruct Nuclio to start processing code only from this location, and then performs basic Nuclio function configuration &mdash; defining the name of the function's container image (`mlrun/ml-models`), the function type (`nuclio`), and some additional package installation commands.

> **Note:** You can add code to define function dependencies and perform additional configuration after the `# nuclio: start-code` marker.

In [ ]:
# nuclio: start-code

<b>Specify function dependencies and configuration<b>

## Function code

In [ ]:
import os
import json
import numpy as np
from v3io import dataplane, common
from datetime import datetime

In [ ]:
def init_context(context):
    v3io_access_key = os.getenv('V3IO_ACCESS_KEY')
    container = os.getenv('CONTAINER')
    feature_table_path = os.getenv('FEATURE_TABLE_PATH')
    feature_list = [v.strip() for v in os.getenv('FEATURE_LIST').split(',')]
    serving_events = [v.strip() for v in os.getenv('SERVING_EVENTS').split(',')]
    output_stream_path = os.getenv('OUTPUT_STREAM_PATH')    
    partition_attr = os.getenv('PARTITION_ATTR')
    enrichment_table_path = os.getenv('ENRICHMENT_TABLE_PATH')
    enrichment_key = os.getenv('ENRICHMENT_KEY')
    
    v3io_client = dataplane.Client(endpoint='http://v3io-webapi:8081', access_key=v3io_access_key)
    
    event_handlers = {'registration': process_registration,
                      'purchase': process_purchase,
                      'bet': process_bet,
                      'win': process_win}
    
    setattr(context, 'v3io_client', v3io_client)
    setattr(context, 'container', container)
    setattr(context, 'feature_table_path', feature_table_path)
    setattr(context, 'feature_list', feature_list)
    setattr(context, 'serving_events', serving_events)
    setattr(context, 'output_stream_path', output_stream_path)
    setattr(context, 'partition_attr', partition_attr)
    setattr(context, 'event_handlers', event_handlers)
    setattr(context, 'enrichment_table_path', enrichment_table_path)
    setattr(context, 'enrichment_key', enrichment_key)
    pass

In [ ]:
def handler(context, event):
    if type(event.body) is dict:
        event_dict = event.body
    else:
        event_dict = json.loads(event.body)
        
    if is_relevant_event(context, event_dict):
        event_type = get_event_type(event_dict)
        context.logger.info(f'Incoming event type: {event_type}')
        
        # python switch-case
        process_func = context.event_handlers.get(event_type)
        context.logger.info(f'Processing event {event_dict}')
        response = process_func(context, event_dict)
        context.logger.info(f'Finished processing with status: {response.status_code} - and response body: {response.body} , event: {event_dict}')
        if event_type in context.serving_events and (200 <= response.status_code < 300) :
            context.logger.info(f'sending event for serving')
            write_to_output_stream(context, event_dict)
    else:
        context.logger.info(f'Not relevant event')
    pass

        
def get_event_type(event):
    return event['event_type']


def is_relevant_event(context, event):
    return get_event_type(event) in context.event_handlers
        
def get_features(context, event):
    user_id = event['user_id']
    features_list = context.feature_list
    resp = context.v3io_client.kv.get(container=context.container, 
                                      table_path=context.feature_table_path,
                                      key=str(user_id),
                                      raise_for_status=dataplane.RaiseForStatus.never)
    
    feat_list = [resp.output.item.get(feat) for feat in features_list]
    feat_list = [0 if v is None else v for v in feat_list]
    return json.dumps({'user_id': user_id, 'inputs': np.array(feat_list).reshape(1,-1).tolist()})


def write_to_output_stream(context, event):
    partition_key = event.get(context.partition_attr)    
    data = get_features(context, event)
    
    record = {'partition_key': str(partition_key), 'data': data }
    resp = context.v3io_client.stream.put_records(container=context.container, 
                                                  stream_path=context.output_stream_path, 
                                                  records=[record], 
                                                  raise_for_status=dataplane.RaiseForStatus.never)
    context.logger.info(f'Sent features for user: {event["user_id"]} to serving stream')
    context.logger.debug(f'Feature values: {data}')


def event_time_to_ts(event_time):
    dt = datetime.strptime(event_time,'%Y-%m-%d %H:%M:%S.%f')
    return datetime.timestamp(dt)


def get_sum_count_mean_var_expr(feature: str, current_value):
    sum_str = f"SET {feature}_sum= if_not_exists({feature}_sum, 0) + {current_value};"
    count_str = f"SET {feature}_count= if_not_exists({feature}_count, 0) + 1;"
    delta_str = f"SET {feature}_delta= {current_value} - if_not_exists({feature}_mean, 0);"
    mean_str = f"SET {feature}_mean= if_not_exists({feature}_mean, 0) + ({feature}_delta / {feature}_count);"
    m2_str = f"SET {feature}_m2= if_not_exists({feature}_m2, 0) + ({feature}_delta * ({current_value} - {feature}_mean));"
    var_str = f"SET {feature}_var= {feature}_m2 / (max(2, {feature}_count)-1) ;"
    expression = sum_str + count_str + delta_str + mean_str + m2_str + var_str
    return expression


def update_features(context, user_id, expression, condition):
    return context.v3io_client.kv.update(container=context.container,
                                         table_path=context.feature_table_path,
                                         key=str(user_id),
                                         condition=condition,
                                         expression=expression,
                                         raise_for_status=dataplane.RaiseForStatus.never)


def enrich_event(context, event):
    if context.enrichment_key in event:
        enrichment_key_value = event[context.enrichment_key]
        resp = context.v3io_client.kv.get(container=context.container, 
                                          table_path=context.enrichment_table_path,
                                          key=str(enrichment_key_value),
                                          raise_for_status=dataplane.RaiseForStatus.never)
        if 200 <= resp.status_code <= 299:
            enriched_event = dict(event, **resp.output.item)
            context.logger.info_with('Event was enriched', enriched_event=enriched_event)
            return enriched_event
        else:
            context.logger.debug_with("Couldn't enrich event", 
                                      enrichment_key_value=enrichment_key_value,
                                      response_status=resp.status_code, 
                                      response_body=resp.body.decode('utf-8'))
            return event
    else:
        return event


def process_registration(context, event):
    user_id = event['user_id']
    
    event = enrich_event(context, event)
    
    features = {'user_id': event['user_id'],
               'registration_date': event['event_time'],
               'date_of_birth': event['date_of_birth'],
               'socioeconomic_idx':  event['socioeconomic_idx'],
               'affiliate_url': event['affiliate_url'],
               'label': event['label']}
    
    response = context.v3io_client.kv.put(container=context.container,
                                          table_path=context.feature_table_path,
                                          key=str(user_id),
                                          attributes=features,
                                          raise_for_status=dataplane.RaiseForStatus.never)
    return response


def process_purchase(context, event):
    user_id = event['user_id']
    event_time = event['event_time']
    event_ts = event_time_to_ts(event_time)
    
    purchase_amount = event['amount']

    first_purchase_ts_str = f"SET first_purchase_ts=if_not_exists(first_purchase_ts, {event_ts});"
    sum_count_mean_var_expr = get_sum_count_mean_var_expr('purchase', purchase_amount)
    
    expression = first_purchase_ts_str + sum_count_mean_var_expr
    condition = f"exists(registration_date) AND (NOT exists(first_purchase_ts) OR first_purchase_ts >= ({event_ts} - 86400 ))"
    
    return update_features(context, user_id, expression, condition)


def process_bet(context, event):
    user_id = event['user_id']
    event_time = event['event_time']
    event_ts = event_time_to_ts(event_time)
    
    bet_amount = event['bet_amount']

    sum_count_mean_var_expr = get_sum_count_mean_var_expr('bet', bet_amount)
    
    expression = sum_count_mean_var_expr
    condition = f"first_purchase_ts >= ({event_ts} - 86400 )"
    
    return update_features(context, user_id, expression, condition)


def process_win(context, event):
    user_id = event['user_id']
    event_time = event['event_time']
    event_ts = event_time_to_ts(event_time)
    
    win_amount = event['win_amount']

    sum_count_mean_var_expr = get_sum_count_mean_var_expr('win', win_amount)
    
    expression = sum_count_mean_var_expr
    condition = f"first_purchase_ts >= ({event_ts} - 86400 )"
    
    return update_features(context, user_id, expression, condition)


The following cell uses the `# nuclio: end-code` marker to mark the end of a Nuclio code section and instruct Nuclio to stop parsing the notebook at this point.<br>
> **IMPORTANT:** Do not remove the end-code cell.

In [ ]:
# nuclio: end-code

Set a dictionary for initializing the environment variables used by the function


## Test locally

In [3]:
import v3io.dataplane
import os
v3io_client = v3io.dataplane.Client()

test_path = os.path.join(os.getcwd(), 'test')

# Create a test target stream
v3io_client = v3io.dataplane.Client()
container = 'users'
output_stream_path = os.path.join(test_path.replace('/User', os.getenv('V3IO_USERNAME')), 'serving-stream')
v3io_client.stream.create(container=container, stream_path=output_stream_path, shard_count=1)

# Create a test enrichment table
enrichment_table_path = os.path.join(test_path.replace('/User', os.getenv('V3IO_USERNAME')), 'enrichment-table')
postcode = 11012
attr = {'postcode': postcode ,'socioeconomic_idx': 3}
v3io_client.kv.put(container=container,
                   table_path=enrichment_table_path,
                   key=str(postcode),
                   attributes=attr)

# Create feature table
feature_table_path = os.path.join(test_path.replace('/User', os.getenv('V3IO_USERNAME')), 'feature-table')
feature_list = ['socioeconomic_idx','purchase_sum','purchase_mean','purchase_count',
                'purchase_var','bet_sum','bet_mean','bet_count',
                'bet_var','win_sum','win_mean','win_count','win_var']


In [4]:
envs = {'V3IO_ACCESS_KEY': os.getenv('V3IO_ACCESS_KEY'),
        'FEATURE_TABLE_PATH': feature_table_path,
        'SERVING_EVENTS': ",".join(['bet','win']),
        'FEATURE_LIST': ",".join(feature_list),
        'CONTAINER': container,
        'OUTPUT_STREAM_PATH': output_stream_path,
        'PARTITION_ATTR': 'user_id',
        'ENRICHMENT_TABLE_PATH': enrichment_table_path,
        'ENRICHMENT_KEY':"postcode"}

In [6]:
import nuclio
for key, value in envs.items():
    os.environ[key] = str(value)
    
reg_event = nuclio.Event(body=f'{{"user_id" : 111111 ,"affiliate_url":"aa.biz", "event_type": "registration", "postcode": {postcode}, "event_time": "2020-07-20 11:00:00","date_of_birth": "1970-03-03", "label":0}}'.encode())
pur_event = nuclio.Event(body=b'{"user_id" : 111111 ,"amount": 3000, "event_type": "purchase", "event_time": "2020-07-20 11:00:00.009"}') 
bet_event = nuclio.Event(body=b'{"user_id" : 111111 ,"bet_amount": 300, "event_type": "bet", "event_time": "2020-07-20 11:00:00.889"}') 
init_context(context)
handler(context, reg_event)
handler(context, pur_event)
handler(context, bet_event)

In [ ]:
# cleanup
!rm -rf {test_path}

## Nuclio Deploy

### Convert code to function

We use MLRun `code_to_function` in order to convert the python code to a Nuclio function. We then set the relevant enrivonment variables and streaming trigger.

In [7]:
from mlrun import code_to_function

fn = code_to_function(name='features',image='mlrun/mlrun',kind='nuclio')
fn.set_envs(envs)

### Deploy

In [8]:
fn.deploy()

> 2021-10-03 08:44:32,808 [info] Starting remote function deploy
2021-10-03 08:44:32  (info) Deploying function
2021-10-03 08:44:32  (info) Building
2021-10-03 08:44:32  (info) Staging files and preparing base images
2021-10-03 08:44:32  (info) Building processor image
2021-10-03 08:44:35  (info) Build complete
2021-10-03 08:44:39  (info) Function deploy complete
> 2021-10-03 08:44:39,378 [info] successfully deployed function: {'internal_invocation_urls': ['nuclio-default-features.default-tenant.svc.cluster.local:8080'], 'external_invocation_urls': ['default-tenant.app.dev8.lab.iguazeng.com:30995']}


'http://default-tenant.app.dev8.lab.iguazeng.com:30995'

## Done